# Preprocess data for analysis

## Get nlp utils functions

In [ ]:
!rm -rf ../nlp_utils_repo
!git clone https://github.com/wzwzeyal/nlp_utils_repo.git ./nlp_utils_repo

## Get the data from a Hebrew-Sentiment-Data git project

The data is divided to token and morph datasets

In [ ]:
!rm -rf ../data/external
!git clone https://github.com/OnlpLab/Hebrew-Sentiment-Data.git ../data/external/OnlpLab

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nlp_utils_repo.nlp_utils import clean_data, remove_words

from nltk.probability import FreqDist
from nltk import word_tokenize
import nltk
nltk.download('punkt')


## Prepare data for sentiment analysis
1. Get the data to a pandas
2. Clean the data
3. Keep the stop words (might be usefull for sentiment analsis)

In [ ]:
ROOT_PATH = '../data/external/OnlpLab/Sentiment_Data'
FIG_SIZE = (20, 15)

In [ ]:
def get_onlp_data(token_or_morph, data_type):
  return pd.read_csv(f'{ROOT_PATH}/{token_or_morph}/{data_type}.tsv', sep='\t')

In [ ]:
train_token_df = get_onlp_data('token', 'train')
val_token_df = get_onlp_data('token', 'dev')

train_morph_df = get_onlp_data('morph', 'train')
val_morph_df = get_onlp_data('morph', 'dev')

In [ ]:
datasets = [
  {"name": "train_token_df", "df": train_token_df},
  {"name": "val_token_df", "df": val_token_df},
  {"name": "train_morph_df", "df": train_morph_df},
  {"name": "val_morph_df", "df": val_morph_df},
]

In [ ]:
codes = {0 : 'pos', 1: 'neg', 2: 'nut'}
palette = {'pos': 'green', 'neg': 'red', 'nut': 'gray'}

In [ ]:
for dataset in datasets:
  df = dataset['df']
  clean_data(df, 'comment', 'comment_clean')
  df['sentiment'] = df.label.map(codes)
  df['comment_clean_len'] = df.comment_clean.str.len()

In [ ]:
train_token_df.sample(5)

In [ ]:
val_token_df.sample(5)

In [ ]:
train_morph_df.sample(5)

In [ ]:
val_morph_df.sample(5)

In [ ]:
datasets[0]['df']

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15,15))#, gridspec_kw={'hspace': 1.2, 'wspace': 1.5})

for ax, dataset in zip(axes.flatten(), datasets):
  sns.histplot(dataset['df'], x='comment_clean_len', hue='sentiment', palette=palette, ax=ax, kde=True)
  ax.set_title(dataset['name'])

plt.show()


In [ ]:
MAX_LEN=128

# for dataset in datasets:
#   df = dataset['df']
#   df = df[df.comment_clean_len < MAX_LEN]

for idx in range(len(datasets)):
  df = datasets[idx]['df']
  datasets[idx]['df'] = df[df.comment_clean_len < MAX_LEN]#  & df.comment_clean_len > 0]

for idx in range(len(datasets)):
  df = datasets[idx]['df']
  datasets[idx]['df'] = df[df.comment_clean_len > 0]#  & df.comment_clean_len > 0]

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=FIG_SIZE)

for ax, dataset in zip(axes.flatten(), datasets):
  df = dataset['df']
  sns.histplot(df, x='comment_clean_len', hue='sentiment', palette=palette, ax=ax, kde=True)
  ax.set_title(dataset['name'])

plt.show()

In [ ]:
SENTIMENT_DATA = '../data/for_sentiment'

!mkdir SENTIMENT_DATA

for dataset in datasets:
  name = dataset['name']
  print(dataset['df'].describe())
  dataset['df'].to_csv(
      f'{SENTIMENT_DATA}/{name}.gz',
       compression = 'gzip')

## Prepare data for unsupervised topic modelling
1. Merge the train dnd test data
2. Remove stop words
3. Remove frequent words (might interfere with the topic modelling)

In [ ]:
TOPIC_DATA = '../data/for_topic'

In [ ]:
!git clone https://github.com/gidim/HebrewStopWords.git

In [ ]:
stop_words = open('./HebrewStopWords/heb_stopwords.txt').read().splitlines()

In [ ]:
def preprocess_for_topic(
    df,
    text_col='comment_clean',
    topic_col='topic',
    stop_words=stop_words,
    nof_most_common=10,
    max_freq=400,
    max_topic_len=256):

    df = df.copy()

    df[topic_col] = df[text_col].map(lambda x: remove_words(x, stop_words))
    word_freq = FreqDist(sum(df[topic_col].map(word_tokenize), [])).most_common(nof_most_common)
    words_to_extract = [word[0] for word in word_freq if word[1] > max_freq]  # 'words that repeats themselves over 500 times
    df[topic_col] = df[topic_col].map(lambda x: remove_words(x, words_to_extract))
    df['topic_len'] = df[topic_col].str.len()
    df = df[df['topic_len'] > 0]
    df = df[df['topic_len'] < max_topic_len]
    return df

In [ ]:
train_token_df.head()

In [ ]:
train_token_df.comment_clean[:10]

In [149]:
all_tokens_df = pd.concat([train_token_df, val_token_df]).reset_index()
all_tokens_df['index'] = all_tokens_df.index
all_tokens_df = preprocess_for_topic(all_tokens_df)
all_tokens_df.to_csv(f'{TOPIC_DATA}/all_tokens_df.gz', compression='gzip')

In [ ]:
all_tokens_df.describe()

In [180]:
all_morphs_df = pd.concat([train_morph_df, val_morph_df]).reset_index()
all_morphs_df['index'] = all_morphs_df.index
all_morphs_df = preprocess_for_topic(all_morphs_df)
all_morphs_df.to_csv(f'{TOPIC_DATA}/all_morphs_df.gz', compression='gzip')

In [ ]:
all_morphs_df.describe()

In [152]:
all_tokens_df.tail(50)

,index,comment,label,comment_clean,sentiment,comment_clean_len,topic,topic_len
6717,6717,ברכות לראובן ריבלין קוויתי ואכן זה התממש כולי ...,0,ברכות לראובן ריבלין קוויתי ואכן זה התממש כולי ...,pos,157,ברכות לראובן קוויתי ואכן התממש כולי אושר בבחיר...,116
6719,6719,ועוד עצה דבר מהלב מהרגש אל האנשים אל כולם אנחנ...,0,ועוד עצה דבר מהלב מהרגש אל האנשים אל כולם אנחנ...,pos,86,עצה מהלב מהרגש לשמוע,20
6720,6720,"ידעתי שתבחר , כי אתה ראוי לזה !! עלה והצלח",0,ידעתי שתבחר כי אתה ראוי לזה עלה והצלח,pos,37,ידעתי שתבחר ראוי עלה והצלח,26
6721,6721,"אשרינו שזכינו לנשיא עממי בעל "" ראש פתוח "" "" ...",0,אשרינו שזכינו לנשיא עממי בעל ראש פתוח לכולם,pos,43,אשרינו שזכינו לנשיא עממי בעל פתוח,33
6722,6722,שבת שלום לך ולבני משפחתך אדון יקר,0,שבת שלום לך ולבני משפחתך אדון יקר,pos,33,ולבני משפחתך אדון יקר,21
6723,6723,בשעה טובה ובמזל טוב לנשיא מדינת ישראל . תחי מד...,0,בשעה טובה ובמזל טוב לנשיא מדינת ישראל תחי מדינ...,pos,53,בשעה ובמזל לנשיא תחי,20
6724,6724,רובי ריבלין - הנשיא הבא של כולנן .,0,רובי ריבלין הנשיא הבא של כולנן,pos,30,כולנן,5
6725,6725,בהצלחה מחר !,0,בהצלחה מחר,pos,10,מחר,3
6726,6726,שבת שלום ובהצלחה אדוני הנשיא .,0,שבת שלום ובהצלחה אדוני הנשיא,pos,28,ובהצלחה אדוני,13
6727,6727,שיהיה לך המון בהצלחה . הבחירה הכי נכונה ונבונה...,0,שיהיה לך המון בהצלחה הבחירה הכי נכונה ונבונה ש...,pos,62,הבחירה נכונה ונבונה שיכולה,26


In [181]:
all_tokens_df['normalize'] = ""
all_morphs_df['normalize'] = ""

In [182]:
from datasets import Dataset

all_morphs_dataset = Dataset.from_pandas(all_morphs_df)

In [183]:
import requests

token = 'F2IGywdQzKHEYiS'


# request = {
#     "token": token,
#     "type": "SEARCH",
#     "sentences": sentences
# }

# print(len(sentences))
# result = requests.post('https://hebrew-nlp.co.il/service/morphology/normalize', json=request).json()


def perform_batch_normalize(examples):
    sentences = list(examples['topic'])

    request = {
    "token": token,
    "type": "SEARCH",
    "sentences": sentences
    }

    result = requests.post('https://hebrew-nlp.co.il/service/morphology/normalize', json=request).json()
    #print(result)
    examples['normalize'] = result
    time.sleep(2.0)
    #all_tokens_df['normalize'] = result
    #print(examples['index'])
    all_morphs_df['normalize'][examples['index']] = result

In [184]:

import time
from ratelimit import limits, RateLimitException, sleep_and_retry


@sleep_and_retry
@limits(calls=1, period=2*60)
def normalize_dataset(input_dataset):
    input_dataset.map(lambda examples: perform_batch_normalize(examples), batch_size=50, batched=True)  
    


In [185]:
%%time
normalize_dataset(all_morphs_dataset)

  0%|          | 0/126 [00:00<?, ?ba/s]/tmp/ipykernel_6170/428493689.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_morphs_df['normalize'][examples['index']] = result
/home/eyalshw/git/ojt_bert/.venv/lib/python3.8/site-packages/pandas/core/internals/blocks.py:937: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)
100%|██████████| 126/126 [05:35<00:00,  2.66s/ba]

CPU times: user 3.79 s, sys: 166 ms, total: 3.96 s
Wall time: 5min 35s


In [169]:
all_tokens_df.iloc[-1]['normalize']

['מַזָּל',
 'טוב',
 'ל',
 'כָּבוֹד',
 'נָשִׂיא',
 'את',
 'בֵּנ',
 'אָדָמ',
 'הכנונ',
 'מָקוֹמ',
 'נָכוֹנ',
 'ישר',
 'כּוֹחַ',
 "ח'אלד",
 'ולי',
 'דְּרוּזִי',
 'גֵּאֶה',
 'צָפוֹנ',
 '.']

In [101]:
all_tokens_df['test'][0:3] = ['aa', 'bb', 'cc']

/tmp/ipykernel_6170/2294139313.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_tokens_df['test'][0:3] = ['aa', 'bb', 'cc']


In [168]:
all_tokens_df.tail()

,index,comment,label,comment_clean,sentiment,comment_clean_len,topic,topic_len,normalize
6767,6767,כבוד הנשיא . אתה חד צדדי . תתאפס .,1,כבוד הנשיא אתה חד צדדי תתאפס,neg,28,חד צדדי תתאפס,13,"[כָּבוֹד, נָשִׂיא, את, חַד, צַד, הִתְאַפֵּס, .]"
6768,6768,"מרגש מאוד , בהצלחה ענקית , אוהבים אותך רובי !!!",0,מרגש מאוד בהצלחה ענקית אוהבים אותך רובי,pos,39,מרגש ענקית,10,"[רֶגֶשׁ, מְאוֹד, הַצְלָחָה, עֲנָקִי, אָהַב, את..."
6769,6769,בימים קשים כולם מקצינים את דעותיהם ומעשיהם מי ...,0,בימים קשים כולם מקצינים את דעותיהם ומעשיהם מי ...,pos,225,בימים קשים מקצינים דעותיהם ומעשיהם יתן ואכן תצ...,166,"[יומ, קָשֶׁה, כל, קָצִינ, אֶת, דֵּעָה, מַעֲשֶֹ..."
6770,6770,שבת שלום רובי ! אני מקווה שבקרוב נשיא המדינה .,0,שבת שלום רובי אני מקווה שבקרוב נשיא המדינה,pos,42,שבקרוב,6,"[שַׁבָּת, שלומ, רַב, אֲנִי, קִוָּה, בקרוב, נָש..."
6771,6771,מזל טוב לך כבוד הנשיא אתה הבן אדם הכנון במקום ...,0,מזל טוב לך כבוד הנשיא אתה הבן אדם הכנון במקום ...,pos,87,הבן הכנון הנכון ישר כוח ח'אלד וולי דרוזי גאה מ...,51,"[מַזָּל, טוב, ל, כָּבוֹד, נָשִׂיא, את, בֵּנ, א..."


In [170]:
all_tokens_df.to_csv(f'{TOPIC_DATA}/all_tokens_df.gz', compression='gzip')

In [177]:
normalize_dataset(all_morphs_dataset)

  0%|          | 0/126 [00:00<?, ?ba/s]/tmp/ipykernel_6170/1371373906.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_morphs_df['normalize'][examples['index']] = result
/home/eyalshw/git/ojt_bert/.venv/lib/python3.8/site-packages/pandas/core/internals/blocks.py:937: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)
100%|██████████| 126/126 [06:19<00:00,  3.01s/ba]


In [178]:
all_morphs_df.to_csv(f'{TOPIC_DATA}/all_morphs_df.gz', compression='gzip')

In [187]:
all_morphs_df.sample(20)

,index,comment,label,comment_clean,sentiment,comment_clean_len,topic,topic_len,normalize
5087,5087,מ אז תמיכתך השבוע ב התבוללות אני מרגיש כי אינך...,1,אז תמיכתך השבוע התבוללות אני מרגיש כי אינך יות...,neg,144,תמיכתך התבוללות מרגיש אינך מייצג אינך חבל תמכת...,61,"[תְּמִיכָה, הִתְבּוֹלְלוּת, הִרְגִּישׁ, אינ, י..."
3279,3279,מחשימם ה ערביים ה אלה הרסו ל אנחנו תחיים אלוהי...,1,מחשימם ערביים אלה הרסו אנחנו תחיים אלוהים יסרט...,neg,59,מחשימם ערביים הרסו תחיים אלוהים יסרטן לוואי,43,"[מחשיממ, עַרְבִי, הָרַס, תחיימ, אֱלֹהִימ, סִרְ..."
6584,6584,עם ישראל חיי מאוחד יותר מתמיד,0,עם ישראל חיי מאוחד יותר מתמיד,pos,29,חיי מאוחד מתמיד,15,"[חַיִּימ, אֻחַד, מתמיד, .]"
5817,5817,איזה שנוי מ רענן - צניעות ו ענווה,0,איזה שנוי רענן צניעות ענווה,pos,27,שנוי רענן צניעות ענווה,22,"[שִׁנּוּי, רעננ, צְנִיעוּת, עֲנָוָה, .]"
6663,6663,נחמה יפה כתבת אני מאד מעריך ו מוקיר את עושרו פ...,0,נחמה יפה כתבת אני מאד מעריך מוקיר את עושרו פוע...,pos,122,נחמה כתבת מעריך מוקיר עושרו פועלו בעליך נקי זך...,78,"[נחמה, כָּתַב, הֶעֱרִיכְ, הוֹקִיר, עֹשֶׁר, פֹּ..."
3282,3282,אנחנו דורשים את רובי כ נשיאנו איש נקה כפיים אה...,0,אנחנו דורשים את רובי נשיאנו איש נקה כפיים אהוב...,pos,65,דורשים נשיאנו איש נקה כפיים אהובעל בריות בריות,46,"[דָּרַשׁ, נָשִׂיא, אִשׁ, נִקָּה, כַּפ, אהובעל,..."
2280,2280,לבכות,0,לבכות,pos,5,לבכות,5,"[בָּכָה, .]"
4304,4304,כל ה כבוד ... חלום אדיר אני חולמת יחד את אתה ו...,0,כל כבוד חלום אדיר אני חולמת יחד את אתה קיווה א...,pos,75,אדיר חולמת יחד קיווה יתגשם נפלא,31,"[אַדִּיר, חָלַמ, יַחַד, קִוָּה, הִתְגַּשֵּׁמ, ..."
5385,5385,נשיא פח ! מוסד מיותר,1,נשיא פח מוסד מיותר,neg,18,פח מוסד מיותר,13,"[פַּח, מוֹסָד, יֻתַּר, .]"
3371,3371,ב ה התחלה פחדו להגיב . בגלל אובמה בגלל ה תקשור...,0,התחלה פחדו להגיב בגלל אובמה בגלל תקשורת נגד יש...,pos,109,התחלה פחדו להגיב אובמה תקשורת ישראת גלת אסורנה...,85,"[הַתְחָלָה, פָּחַד, הֵגִיב, אובמה, תִּקְשֹׁרֶת..."
